# Lab 11

In [ ]:
import random
from tkinter import *
import requests

url = "https://the-trivia-api.com/api/questions?categories=film_and_tv&limit=10&region=US&difficulty=easy"
get_request = requests.get(url).json()

root = Tk()
root.title("Trivia")
root.geometry("1000x800")

canvas = Canvas(root, width=800, height=250, bg="khaki1")
canvas.place(relx=.5, rely=.5, anchor= CENTER)

game_display = canvas.create_text(200, 75, text="", fill="black", font=('Helvetica 15 bold'))

feedback = Label(root, text="", pady=10, font=("arial", 15, "bold"))

user_answer = StringVar()
user_answer.set("")

score = 0
question_counter = 0

def generate_qlist(json):
    question_list = []
    for i in json:
        i = i["question"]
        question_list.append(i)
    return question_list

def gen_question(list_):
    choice = random.choice(list_)
    return choice

def generate_answers(json, question):
    option_list = []
    for i in get_request:
        if i["question"] == question:
            cor = i["correctAnswer"]
            option_list.append(cor)
            for incor in i["incorrectAnswers"]:
                option_list.append(incor)
    random.shuffle(option_list)
    return option_list

def ask_question(question, answers):
    canvas.itemconfig(game_display, text=question, width=300)

def gen_radiobuttons(answers):
    y_pos = 100
    for i in answers:
        rad_btn = Radiobutton(canvas, text=i, variable=user_answer, value=i, font=('Helvetica 10 bold'))
        rad_btn.place(x=50, y=y_pos)
        y_pos += 20

def clear_radiobuttons():
    for widget in canvas.winfo_children():
        if isinstance(widget, Radiobutton):
            widget.destroy()

def eval_score():
    global question_counter
    answer = user_answer.get()
    for i in get_request:
        if i["question"] == current_question:
            if i["correctAnswer"] == answer:
                global score
                score += 10
                feedback["text"] = f"Correct! You have {score} points."
                feedback.place(x=400, y=500)
            else:
                feedback["text"] = f"Incorrect. You have {score} points."
                feedback.place(x=400, y=500)
    question_counter += 1

def next_question():
    eval_score()
    if question_counter < 10:
        clear_radiobuttons()
        user_answer.set("")
        main()
    else:
        feedback["text"] = f"Game Over! You scored {score} points! You got {score // 10} questions correct."
        feedback.place(x=350, y=500)
        next_button["state"] = DISABLED
        
def main():
    q_list = generate_qlist(get_request)
    global current_question
    current_question = gen_question(q_list)
    ans_list = generate_answers(get_request, current_question)
    ask_question(current_question, ans_list)
    gen_radiobuttons(ans_list)

main()

next_button = Button(root, text="Next", command=next_question)
next_button.place(x=150, y=470)

root.mainloop()